This notebook contains the code in data scraping.

In [27]:
# Import the dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import json
import time
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from config import gkey

In [15]:
# Launches browser to search
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Kevin\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [16]:
dojo_list = []

# Visits the url
url = "https://dojos.info/NewJersey/Counties.aspx"
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")
time.sleep(2)


# Searches the website for the link to the counties
browser.links.find_by_partial_text("County").click()
time.sleep(1)

if browser.url == "https://dojos.info/NewJersey/Counties.aspx#google_vignette":
    browser.visit(url)
    browser.links.find_by_partial_text("County").click()

response = requests.get(browser.url)
soup1 = bs(response.text,'html.parser')

uList = soup1.find('ul',class_ = 'dojolist')
dojolist = uList.find_all('h3')
dojolen = len(dojolist)


# Selects the dojo list to click.
ulist = browser.find_by_css('.dojolist')

h3 = ulist.find_by_tag('h3')

# Click the the link to move to the dojo information.
a_tags = h3.find_by_tag('a')

a_tags.click()

for x in range(1,dojolen + 1):

    # Finds the address for the dojo.
    address = browser.find_by_tag('address')

    # This will pull the address text
    p_address = address.text

    # This will split the text string into usable pieces.
    recreate = p_address.replace('\n'," | ").split("|")
    
    if len(recreate) > 3:
        recreate.pop(0)
    
    # This is the street address
    street = recreate[0].split()
    streets = " ".join(street)
    split = recreate[1].split(',')

    # The city address
    city = split[0]

    # Further splits the string into the state and zipcode parts.
    split2 = split[1].split(" ")
    state = split2[1]
    zipcode = split2[2]

    # This will pull the name of the dojo.
    dojo_name = browser.find_by_css('#pageTop')
    dojo = dojo_name.find_by_tag("h1").text

    # This will find the styles that the dojo teaches
    response = requests.get(browser.url)
    soup2 = bs(response.text , "html")

    style = []

    styles = soup2.find('ul', class_ = 'styles')
    for dank in styles.find_all('li'):
        style.append(dank.text)

    # Finding the phone number.
    phone = browser.find_by_css('.phone')
    dojo_phone = phone.text.split(" ")[1]

    dojo_dict = {'Name': dojo,
                'Phone': dojo_phone,
                'Zipcode': zipcode,
                'Street': streets,
                'City': city,
                'State': state,
                'Style': style}
    
    dojo_list.append(dojo_dict)
    browser.back()
    time.sleep(1)
    
    try:
        ulist = browser.find_by_css('.dojolist')
        h3 = ulist.find_by_tag('h3')[x]
        h3.find_by_tag("a").click()

    except:
        print('done')

done


In [17]:
# Quits out of the browser.
browser.quit()

In [25]:
call_list = []
for rows in range(len(dojo_list)):
    strt = dojo_list[rows]['Street']
    city = dojo_list[rows]['City']
    state = dojo_list[rows]['State']
    key = 'dank'
    api_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={strt},{city},+{state}&key={gkey}'
    api_url
    call_list.append(api_url.replace(" ","+"))

In [29]:
geo_data = requests.get(call_list[0]).json()
print(geo_data)

{'results': [{'address_components': [{'long_name': 'Unit 13 A', 'short_name': 'Unit 13 A', 'types': ['subpremise']}, {'long_name': '2511', 'short_name': '2511', 'types': ['street_number']}, {'long_name': 'Fire Road', 'short_name': 'Fire Rd', 'types': ['route']}, {'long_name': 'Egg Harbor Township', 'short_name': 'Egg Harbor Township', 'types': ['locality', 'political']}, {'long_name': 'Atlantic County', 'short_name': 'Atlantic County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'New Jersey', 'short_name': 'NJ', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}, {'long_name': '08234', 'short_name': '08234', 'types': ['postal_code']}, {'long_name': '5650', 'short_name': '5650', 'types': ['postal_code_suffix']}], 'formatted_address': '2511 Fire Rd Unit 13 A, Egg Harbor Township, NJ 08234, USA', 'geometry': {'location': {'lat': 39.4209654, 'lng': -74.5275233}, 'location_ty

In [30]:
print(json.dumps(geo_data, indent=4, sort_keys=True))

{
    "results": [
        {
            "address_components": [
                {
                    "long_name": "Unit 13 A",
                    "short_name": "Unit 13 A",
                    "types": [
                        "subpremise"
                    ]
                },
                {
                    "long_name": "2511",
                    "short_name": "2511",
                    "types": [
                        "street_number"
                    ]
                },
                {
                    "long_name": "Fire Road",
                    "short_name": "Fire Rd",
                    "types": [
                        "route"
                    ]
                },
                {
                    "long_name": "Egg Harbor Township",
                    "short_name": "Egg Harbor Township",
                    "types": [
                        "locality",
                        "political"
                    ]
                },
              

In [20]:
df = pd.DataFrame(dojo_list)
df

,Name,Phone,Zipcode,Street,City,State,Style
0,Cardo Urso's HABU PIT,609-415-5014,08234,"2511 Fire Road, Unit 13 A",Egg Harbor Township,NJ,"[Boxing, Jiu-Jitsu (Jujutsu/Jujitsu), Judo, Mi..."
1,Lightning Lyall MMA,609-474-3055,08406,Ventnor Ave,Ventnor City,NJ,"[Karate, Kick Boxing, Mixed Martial Arts (MMA)..."
2,Lightning MMA,609-474-3055,08406,Newport Ave,Ventnor City,NJ,"[Kick Boxing, Mixed Martial Arts (MMA), Muay T..."
3,South Jersey Wing Chun Kuen Do,609-432-6156,08232,801 N Main Street,Pleasantville,NJ,"[Kung Fu, Wing Chun]"
4,Ultimate Martial Arts Academy,609-625-0880,08330,4450 E Black Horse Pike,Hamilton Township,NJ,"[Hapkido, Karate, Tae Kwon Do]"
5,Devine's Martial Arts,609-839-5981,08330,4450 Black Horse Pike,Mays Landing,NJ,[Tang Soo Do]
6,Hammonton Karate,888-986-4652,08037,1170 White horse pike,Hammonton,NJ,"[Arnis/Kali/Escrima/FMA, Jeet Kune do, Karate,..."
7,"Rich Miller, Sr.'s Dragon Kenpo Karate",609-204-5019,08330,84 Putters Lane,Mays Landing,NJ,"[Goju Ryu Karate, Jeet Kune do, Judo, Mixed Ma..."
8,Shudokan School of Karate Hammonton,609-668-1189,08037,29 South Washington St.,Hammonton,NJ,"[Capoeira, Karate, Kick Boxing, Kobudo]"
9,Hammonton Martial Arts,609-214-7888,08037,1170 White Horse Pike,Hammonton,NJ,"[Karate, Kick Boxing]"


In [13]:
dojo_list

[{'Name': "Cardo Urso's HABU PIT",
  'Phone': '609-415-5014',
  'Zipcode': '08234',
  'Street': ['2511', 'Fire', 'Road,', 'Unit', '13', 'A'],
  'City': ' Egg Harbor Township',
  'State': 'NJ',
  'Style': ['Boxing',
   'Jiu-Jitsu (Jujutsu/Jujitsu)',
   'Judo',
   'Mixed Martial Arts (MMA)',
   'Muay Thai',
   'Sambo']},
 {'Name': 'Lightning Lyall MMA',
  'Phone': '609-474-3055',
  'Zipcode': '08406',
  'Street': ['Ventnor', 'Ave'],
  'City': ' Ventnor City',
  'State': 'NJ',
  'Style': ['Karate',
   'Kick Boxing',
   'Mixed Martial Arts (MMA)',
   'Muay Thai',
   'Privates',
   'Group']},
 {'Name': 'Lightning MMA',
  'Phone': '609-474-3055',
  'Zipcode': '08406',
  'Street': ['Newport', 'Ave'],
  'City': ' Ventnor City',
  'State': 'NJ',
  'Style': ['Kick Boxing', 'Mixed Martial Arts (MMA)', 'Muay Thai']},
 {'Name': 'South Jersey Wing Chun Kuen Do',
  'Phone': '609-432-6156',
  'Zipcode': '08232',
  'Street': ['801', 'N', 'Main', 'Street'],
  'City': ' Pleasantville ',
  'State': 'NJ',


In [ ]:
# Selects the dojo list to click.
ulist = browser.find_by_css('.dojolist')

# Click the the link to move to the dojo information.
ulist.find_by_tag('a')

ulist

In [ ]:
# Visits the url
url = "https://dojos.info/NewJersey/Counties.aspx"
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")

In [ ]:
# Launches browser to search
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
dojo_list = []

# Visits the url
url = "https://dojos.info/NewJersey/Counties.aspx"
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")
time.sleep(2)

In [ ]:
# Setting the url
url = "https://dojos.info/NewJersey/Counties.aspx"

# Creating the response
response = requests.get(url)

# Using beautiful soup
soup = bs(response.text,"html.parser")

In [ ]:
# Code to create a list of counties.

county_tag = []
state = soup.find('ul', class_ = "olist")
county = state.find_all("a")
for i in range(len(county)):
    county_tag.append(county[i].text)

In [ ]:
county = []
for j in county_tag:
    counties = j.replace(" ","")
    county_url = f"https://dojos.info/NewJersey/{counties}"
    county.append(county_url)

In [ ]:
url2 = county[0]
response = requests.get(url2)
soup = bs(response.text,"html.parser")
dank = []
for ulist in soup.find_all('ul', class_ = 'dojolist'):
    for h3 in ulist.find_all('h3'):
        for a in h3.find_all('a'):
            dank.append(a['href'])

In [ ]:
dank

In [ ]:
dojo = []
for k in dank:
    dojo_url = f"https://dojos.info{k}"
    dojo.append(dojo_url)

In [ ]:
url3 = dojo[0]
response = requests.get(url3)
soup = bs(response.text,"html.parser")

address = soup.find("address")
new_address = address.get_text("|").split("|")
street1 = new_address[0].replace("\r\n\t\t","")
street2 = street1.split()
street = " ".join(street2)
street


In [ ]:
new_address

In [ ]:
# Finds the address for the dojo.
address = browser.find_by_tag('address')

# This will pull the address text
p_address = address.text

# This will split the text string into usable pieces.
recreate = p_address.replace('\n'," | ").split("|")
street = recreate[0]
split = recreate[1].split(',')
city = split[0]

split2 = split[1].split(" ")
state = split2[1]
zipcode = split2[2]

# This will pull the name of the dojo.
dojo_name = browser.find_by_css('#pageTop')
dojo = dojo_name.find_by_tag("h1").text

# Finding the phone number.
phone = browser.find_by_css('.phone')
dojo_phone = phone.text.split(" ")[1]

In [ ]:
county

In [ ]:
county_url2 = 'https://dojos.info/NewJersey/AtlanticCounty/'
response = requests.get(county_url2)

soup = bs(response.text,"html.parser")
ulist = soup.find('ul', class_ ='dojolist')

dojo_link = ulist.find('h3')
dojo_link

In [ ]:
recreate

In [ ]:
response = requests.get(browser.url)
soup = bs(response.text , "html")

style = []

styles = soup.find('ul', class_ = 'styles')
for dank in styles.find_all('li'):
    style.append(dank.text)
style

The Code below is to start pulling information on the dojo

In [ ]:
url2 = 'https://dojos.info/Acmfs/'
response = requests.get(url2)

# Using beautiful soup
soup = bs(response.text,"html.parser")

In [ ]:
address = soup.find('address')
address

In [ ]:
parts = address.find_all("a")

In [ ]:
for i in range(3):
    print(parts[i].text)

In [ ]:
part = soup.find('address', id='text')

In [ ]:
# Finding the phone number.
phone = browser.find_by_css('.phone')
dojo_phone = phone.text.split(" ")
dojo_phone[1]

In [ ]:
address.text

In [ ]:
address

Pulls the Address

In [ ]:
new_address = address.get_text("|")

In [ ]:
new_address.replace(" ","")

In [ ]:
first = new_address.split("|")[0]

In [ ]:
new_address.split("|")

In [ ]:
new_address.replace("\r\n\t\t","")

In [ ]:
second = first.replace("\r\n\t\t","")

In [ ]:
# Splits and joins the string.
third = second.split()
fourth = " ".join(third)
fourth

In [ ]:
dojo_name = soup.find("div", id ="pageTop")
dojo_name.find("h1").get_text()

In [ ]:
url2 = 'https://dojos.info/Acmfs/'
response = requests.get(url2)

# Using beautiful soup
soup = bs(response.text,"html.parser")

address = soup.find('address')
new_address = address.get_text("|")
clean_address = new_address.replace("\r\n\t\t","")
parts = clean_address.split("|")

In [ ]:
parts

In [ ]:
address.get_text("|")\
    .replace("\r\n\t\t","")\
    .split("|")